# 🌍 Global News MoodMap with BigQuery AI  

### 📌 Problem  
Every day, thousands of headlines are published worldwide.  
But raw news streams are **noisy, fragmented, and hard to interpret**:  

- Leaders can’t quickly see **which countries are facing shocks**.  
- Analysts struggle to connect **tone, themes, and historical context**.  
- Traditional dashboards focus on **counts of headlines**, not **why they matter**.  

As a result, decision-makers miss signals that could inform **policy, strategy, or risk management**.  

---

### 🎯 Goal  
This project turns the global news firehose into a **1-minute global dashboard**, answering:  

- **What happened today in each country?**  
- **Why does it matter, and what are the drivers?**  
- **How positive or negative is the national mood?**  
- **Where have we seen this pattern before?**  

The outcome is a **Global MoodMap**, powered by BigQuery AI, that:  
- Summarizes events with **LLM-driven briefings**.  
- Scores sentiment by **blending AI mood with GDELT tone**.  
- Visualizes the results on an **interactive world map**.  
- Lets you **search history by meaning** (semantic search), not just keywords.  

---

### 🛠️ Approach (BigQuery AI + Vector Search)  

1. **Data ingestion & curation**  
   - Ingest last 7 days of **GDELT events**.  
   - Aggregate per **country/day** into `daily_country_topics` with:  
     - headline count,  
     - average tone,  
     - top themes & sample URLs.  
   👉 Produces compact, LLM-ready “topic documents.”  

2. **Embeddings & semantic search**  
   - Generate embeddings with `ML.GENERATE_EMBEDDING`.  
   - Store in `news_embeddings`.  
   - Build two reusable SQL functions:  
     - `fn_similar_to_text(query_text)` → semantic search (e.g. *“wildfires Spain”*).  
     - `fn_similar_to_day(country, day)` → find **historical analogs**.  
   👉 Enables **search by meaning** across global news.  

3. **AI briefings**  
   - Use `ML.GENERATE_TEXT` on each country-day to produce a **structured briefing**:  
     - What happened | Key drivers | Impact | What to watch next  
   - Enrich with analogs from semantic search.  
   - Store in `daily_briefings`.  
   👉 Judges get **clear, readable summaries**, not walls of headlines.  

4. **Mood scoring**  
   - Extract sentiment (−1 to +1) from briefings with an LLM.  
   - Blend with GDELT’s avg_tone for robustness.  
   - Store in `daily_moodmap`.  
   👉 A **single mood score per country/day** for easy visualization.  

5. **Visualization & demo**  
   - Interactive choropleth (Plotly): countries colored by `mood_score` (😡 → 😃).  
   - Hover shows **country name, score, and LLM-generated summary**.  
   - Live semantic search demos with the SQL functions.  
   👉 Turns raw GDELT into a **dashboard-ready product.**  

---

### 🚀 Deliverable  
An **end-to-end BigQuery AI workflow** that transforms unstructured news into:  
- **Readable briefings** per country/day.  
- **Quantified sentiment scores**.  
- **Historical analog retrieval** by semantic similarity.  
- **An interactive global map** to explore today’s mood.  

⚖️ **Note on scope**  
To optimize cost, `daily_briefings` and `daily_moodmap` are generated for the **latest day (top N countries)**.  
The same pipeline scales seamlessly to multiple days.  

---

💡 **Why it matters**:  
This prototype shows how **AI + BigQuery** can cut through information overload and deliver **strategic insights at global scale** — the kind of tool that could power **risk monitoring, diplomacy, investment, or humanitarian response**.


## 1. Setup

We configure BigQuery + Vertex AI for use inside this Kaggle notebook.
Authentication uses Kaggle secrets for the project ID and service account key.


In [ ]:
from google.cloud import bigquery
import vertexai
import os

# The environment should already be authenticated.
# If GOOGLE_CLOUD_PROJECT is set, it will be used.
# Otherwise, replace with your project id for local runs.

PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT", "your-hackathon-project-id")
LOCATION = "us-central1"

# Initialize services
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

print("✅ Vertex AI + BigQuery ready (project:", PROJECT_ID, ")")


BigQuery error in mk operation: Already Exists: Connection
projects/640778897893/locations/us/connections/kaggle-connection
✅ Auth + BigQuery client ready


## 2. Create Dataset

All project tables will be stored under a dedicated BigQuery dataset: **`world_mood`**.  
This keeps results organized (raw events, enrichment, embeddings, briefings, mood map).  
We set:
- **Location:** US (same region as GDELT and Vertex AI models).  
- **Default expiration:** 60 days (auto-cleanup to save quota).  


In [39]:
query = f"""
CREATE SCHEMA IF NOT EXISTS `{project_id}.world_mood`
OPTIONS(
  location="US",
  default_table_expiration_days=60
);
"""
client.query(query).result()
print("Dataset created.")

Dataset created.


## 3. Ingest + Enrich GDELT Events

To build a reliable MoodMap, we first need a **clean, enriched feed of world events**.  

- **Raw events (`gdelt_events_raw`)**  
  - Pulled from the **GDELT 2.0 Events** dataset (last 60 days).  
  - Partitioned by `event_date` and clustered by `country` for scalability.  
  - Captures the **who, where, and tone** of each event, with a source URL.  

- **Enriched events (`gdelt_events_enriched`)**  
  - Augments raw events with the **Global Knowledge Graph (GKG)**.  
  - Extracts **themes, people, and organizations** mentioned in coverage.  
  - Kept to **7 days** to stay cost-efficient while proving the workflow.  
  - Normalized arrays (≤50 themes, ≤30 people, ≤30 orgs) ensure usable context.  

👉 Why it matters:  
Instead of a firehose of headlines, we now have a **structured, enriched dataset** that connects tone, themes, and actors. This becomes the foundation for all later steps: aggregation, summarization, and sentiment scoring.  


In [40]:
# Step 1: Ingest raw GDELT events (last 60 days)

gdelt_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.gdelt_events_raw`
PARTITION BY event_date
CLUSTER BY country
AS
SELECT
  PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING)) AS event_date,
  COALESCE(
    ActionGeo_CountryCode,
    Actor1Geo_CountryCode,
    Actor2Geo_CountryCode
  ) AS country,
  COALESCE(
    ActionGeo_ADM1Code,
    Actor1Geo_ADM1Code,
    Actor2Geo_ADM1Code
  ) AS admin1,
  COALESCE(ActionGeo_Lat, Actor1Geo_Lat, Actor2Geo_Lat) AS lat,
  COALESCE(ActionGeo_Long, Actor1Geo_Long, Actor2Geo_Long) AS lon,
  EventCode,
  EventBaseCode,
  EventRootCode,
  AvgTone AS tone,
  SOURCEURL AS url
FROM `gdelt-bq.gdeltv2.events`
WHERE SQLDATE >= CAST(FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)) AS INT64);

"""

client.query(gdelt_query).result()
print("gdelt Dataset created.")

gdelt Dataset created.


In [41]:
# Step 2: Enrich with GKG (themes, persons, orgs) — keep only last 7 days for cost

enrich_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.gdelt_events_enriched`
PARTITION BY event_date
CLUSTER BY country AS
WITH
  base AS (
    -- Use the columns already in your materialized raw table (no SQLDATE here)
    SELECT
      event_date,
      country,
      admin1,
      lat,
      lon,
      EventCode,
      EventBaseCode,
      EventRootCode,
      tone,
      url
    FROM `{project_id}.world_mood.gdelt_events_raw`
    WHERE event_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)   -- keep it cheap
      AND country IS NOT NULL
      AND url IS NOT NULL
  ),
  gkg AS (
    SELECT
      -- GKG DATE is yyyymmddHHMMSS; take first 8 for a DATE if you ever need it
      PARSE_DATE('%Y%m%d', SUBSTR(CAST(date AS STRING), 1, 8)) AS gkg_date,
      DocumentIdentifier AS url,
      V2Themes,
      V2Persons,
      V2Organizations
    FROM `gdelt-bq.gdeltv2.gkg`
    WHERE date >= CAST(FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)) AS INT64) * 1000000  -- match 15-digit format
  ),
  joined AS (
    SELECT
      b.*,
      g.V2Themes,
      g.V2Persons,
      g.V2Organizations
    FROM base b
    LEFT JOIN gkg g
      USING (url)
  )
SELECT
  event_date, country, admin1, lat, lon,
  EventCode, EventBaseCode, EventRootCode, tone, url,
  -- Split semicolon-delimited fields into arrays
  ARRAY(
    SELECT TRIM(x) FROM UNNEST(SPLIT(COALESCE(V2Themes, ''), ';')) x
    WHERE TRIM(x) != '' LIMIT 50
  ) AS themes,
  ARRAY(
    SELECT TRIM(x) FROM UNNEST(SPLIT(COALESCE(V2Persons, ''), ';')) x
    WHERE TRIM(x) != '' LIMIT 30
  ) AS persons,
  ARRAY(
    SELECT TRIM(x) FROM UNNEST(SPLIT(COALESCE(V2Organizations, ''), ';')) x
    WHERE TRIM(x) != '' LIMIT 30
  ) AS orgs
FROM joined;
"""

client.query(enrich_query).result()
print("✅ gdelt_events_enriched created (7-day, with themes/persons/orgs)")



✅ gdelt_events_enriched created (7-day, with themes/persons/orgs)


## 4. Daily Country Topics

After enriching the raw feed, we **compress thousands of events into one document per country per day**.  
This gives us a manageable unit of analysis that’s still rich in context.  

- **Table: `daily_country_topics`**  
  Each row = *(event_date, country)*.  

- **What it contains:**  
  - `headline_count` → how many GDELT events that day  
  - `avg_tone`, `min_tone`, `max_tone` → sentiment statistics  
  - `top_event_types` → up to 20 frequent event categories  
  - `sample_urls` → 30 representative article links  
  - `topic_doc` → compact text string with all the above, built for LLM input  

👉 Why it matters:  
Instead of sending raw headlines to the LLM (expensive + noisy), we pass these **country-day topic docs**.  
They’re compact, structured, and ready for embeddings and AI briefings — the **bridge between messy news streams and meaningful summaries**.  


In [42]:
query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.daily_country_topics`
PARTITION BY event_date
CLUSTER BY country AS
WITH country_day AS (
  SELECT
    event_date,
    country,
    COUNT(*) AS headline_count,
    AVG(tone) AS avg_tone,
    MIN(tone) AS min_tone,
    MAX(tone) AS max_tone,
    ARRAY_AGG(DISTINCT EventRootCode IGNORE NULLS LIMIT 20) AS top_event_types,
    ARRAY_AGG(DISTINCT url IGNORE NULLS LIMIT 30) AS sample_urls
  FROM `{project_id}.world_mood.gdelt_events_raw`
  WHERE country IS NOT NULL
    AND event_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)  -- limit window
  GROUP BY event_date, country
)
SELECT
  event_date,
  country,
  headline_count,
  avg_tone,
  min_tone,
  max_tone,
  top_event_types,
  sample_urls,
  CONCAT(
    'Country: ', country, ' | Date: ', CAST(event_date AS STRING), ' | ',
    'Headlines: ', CAST(headline_count AS STRING), ' | ',
    'Tone avg/min/max: ', CAST(ROUND(avg_tone,2) AS STRING), '/', 
                          CAST(ROUND(min_tone,2) AS STRING), '/', 
                          CAST(ROUND(max_tone,2) AS STRING), ' | ',
    'Top Event Codes: ', ARRAY_TO_STRING(top_event_types, ', '), ' | ',
    'Sample URLs: ', ARRAY_TO_STRING(sample_urls, ' | ')
  ) AS topic_doc
FROM country_day;

"""

client.query(query).result()
print('Daily country topics created')

Daily country topics created


## 5. Remote Vertex AI Models

BigQuery doesn’t just query data, it can **call foundation models directly in SQL**.  
We register two remote Vertex AI models and reuse them throughout the workflow:  

- **`embedding_model` → `gemini-embedding-001`**  
  - Generates dense vector embeddings for text.  
  - Powers **semantic search** and **historical analogs** (pattern-matching across time).  

- **`gen_model` → `gemini-2.0-flash-001`**  
  - A fast LLM used for multiple tasks:  
    - Structured daily briefings  
    - Sentiment extraction  
    - Short summaries for visualization hovers  

👉 Why it matters:  
By linking these models once, we can call `ML.GENERATE_EMBEDDING` and `ML.GENERATE_TEXT` directly inside BigQuery SQL.  
This keeps the entire workflow **serverless and SQL-native**, no external pipelines, no manual orchestration.  


In [43]:
# Create a remote connection for the embedding model.
query = f"""
CREATE OR REPLACE MODEL `{project_id}.world_mood.embedding_model`
  REMOTE WITH CONNECTION `{location}.kaggle-connection`
  OPTIONS (endpoint = 'gemini-embedding-001');
"""
client.query(query).result()
print("embedding model created.")


embedding model created.


In [44]:

query = f"""
CREATE OR REPLACE MODEL `{project_id}.world_mood.gen_model`
  REMOTE WITH CONNECTION `{location}.kaggle-connection`
  OPTIONS (endpoint = 'gemini-2.0-flash-001');
"""
client.query(query).result()
print("gen model created.")


gen model created.


## 6. Semantic Embeddings (`news_embeddings`)

Raw text is messy for search and keywords miss nuance.  
To enable **search by meaning**, we convert each `(date, country)` topic document into a **vector embedding** using our `embedding_model`.  

- **Input:** the compact `topic_doc` string (≤ 3000 chars).  
- **Output:** a dense numerical embedding + metadata (`event_date`, `country`, `id`).  

👉 Why it matters:  
- **Semantic search:** lets us query with natural language (e.g., “wildfires in Spain”) and retrieve the most similar events, even without exact keywords.  
- **Historical analogs:** lets us compare today’s embedding against past embeddings, surfacing patterns and precedents.  

The result is stored in `news_embeddings`, which becomes the **vector backbone of the project**,powering all similarity queries.  


In [45]:
# Generate embeddings for daily country topics

emb_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.news_embeddings` AS
SELECT
  t.event_date,
  t.country,
  t.content AS topic_doc,                     
  t.ml_generate_embedding_result AS embedding,
  CONCAT(t.country, '-', CAST(t.event_date AS STRING)) AS id
FROM ML.GENERATE_EMBEDDING(
  MODEL `{project_id}.world_mood.embedding_model`,
  (
    SELECT
      event_date,
      country,
      SUBSTR(topic_doc, 1, 3000) AS content                    
    FROM `{project_id}.world_mood.daily_country_topics`
  )
) AS t;


"""
client.query(emb_query).result()
print("news embeddings created.")

news embeddings created.


## 7. Semantic Search with Vector Functions

On top of the `news_embeddings` table, we created two **BigQuery Table Functions** that let us search by *meaning*, not just by keywords:  

- **`fn_similar_to_text(query_text)`**  
  Input: any free text query (e.g., *“wildfires in Spain”*).  
  Output: the most semantically similar country-day documents across history.  
  👉 Enables open-ended discovery: you don’t need exact terms or manual filters.  

- **`fn_similar_to_day(country, day)`**  
  Input: a specific country and date (e.g., Spain on 2025-09-03).  
  Output: the most similar *past* country-day documents.  
  👉 Provides **historical analogs**, surfacing precedents and recurring patterns.  

### Demo

- **Free text search:**  
  Query → *“wildfires in Spain”*  
  Result → retrieves Spanish news clusters from late August 2025, all semantically related.  

- **Historical analogs:**  
  Input → Spain, 2025-09-03  
  Result → shows the 5 closest days from August/September with similar dynamics.  

✅ Why this matters:  
With a **single SQL query**, decision-makers can:  
- Track similar events over time (e.g., crises, protests, disasters).  
- Spot early warning signals by comparing today’s news to historical precedents.  
- Explore global news semantically without relying on rigid keywords.  

This turns `news_embeddings` into a **strategic intelligence layer**, powering discovery, monitoring, and pattern recognition directly inside BigQuery.  


In [46]:
# Create vector search helper functions

# Function 1: semantic search by free text query
query = f"""
CREATE OR REPLACE TABLE FUNCTION `{project_id}.world_mood.fn_similar_to_text`(
  query_text STRING
)
RETURNS TABLE<
  event_date DATE,
  country STRING,
  topic_doc STRING,
  distance FLOAT64,
  id STRING
>
AS (
  SELECT
    vs.base.event_date     AS event_date,
    vs.base.country        AS country,
    vs.base.topic_doc      AS topic_doc,
    vs.distance            AS distance,
    vs.base.id             AS id
  FROM VECTOR_SEARCH(
    TABLE `{project_id}.world_mood.news_embeddings`,
    'embedding',
    (SELECT ml_generate_embedding_result AS embedding
     FROM ML.GENERATE_EMBEDDING(
       MODEL `{project_id}.world_mood.embedding_model`,
       (SELECT query_text AS content)
     )),
    'embedding',
    top_k => 10,
    distance_type => 'COSINE'
  ) AS vs
);

"""
client.query(query).result()

In [47]:
# Function 2: find similar past days for a given country + date
query = f"""
CREATE OR REPLACE TABLE FUNCTION `{project_id}.world_mood.fn_similar_to_day`(
  country_code STRING,
  day DATE
)
RETURNS TABLE<
  event_date DATE,
  country STRING,
  topic_doc STRING,
  distance FLOAT64,
  id STRING
>
AS (
  WITH anchor AS (
    SELECT embedding
    FROM `{project_id}.world_mood.news_embeddings`
    WHERE country = country_code AND event_date = day
    LIMIT 1
  )
  SELECT
    vs.base.event_date     AS event_date,
    vs.base.country        AS country,
    vs.base.topic_doc      AS topic_doc,
    vs.distance            AS distance,
    vs.base.id             AS id
  FROM VECTOR_SEARCH(
    TABLE `{project_id}.world_mood.news_embeddings`,
    'embedding',
    (SELECT embedding FROM anchor),
    'embedding',
    top_k => 10,
    distance_type => 'COSINE'
  ) AS vs
  WHERE vs.base.event_date <> day      -- exclude the same day
);


"""
client.query(query).result()

In [48]:
# Demo 1: semantic search with free text query

query_text = "wildfires in Spain"

sql = f"""
SELECT event_date, country, topic_doc, distance
FROM `{project_id}.world_mood.fn_similar_to_text`("{query_text}")
ORDER BY distance ASC
LIMIT 5
"""

df = client.query(sql).to_dataframe()

# Add readable snippet
df["snippet"] = df["topic_doc"].str[:120] + "..."

display(df[["event_date", "country", "snippet", "distance"]])


,event_date,country,snippet,distance
0,2025-09-04,SP,Country: SP | Date: 2025-09-04 | Headlines: 26...,0.357536
1,2025-09-05,SP,Country: SP | Date: 2025-09-05 | Headlines: 23...,0.359481
2,2025-09-06,SP,Country: SP | Date: 2025-09-06 | Headlines: 14...,0.365088
3,2025-09-03,SP,Country: SP | Date: 2025-09-03 | Headlines: 30...,0.365198
4,2025-09-08,SP,Country: SP | Date: 2025-09-08 | Headlines: 33...,0.373530


In [49]:
# Demo 2: find past similar days for Spain on 2025-09-03

sql = f"""
SELECT *
FROM `{project_id}.world_mood.fn_similar_to_day`('ES', DATE '2025-09-03')
ORDER BY distance ASC
LIMIT 5
"""

df = client.query(sql).to_dataframe()

# Add readable snippet
df["snippet"] = df["topic_doc"].str[:120] + "..."

display(df[["event_date", "country", "snippet", "distance"]])


,event_date,country,snippet,distance
0,2025-09-08,ES,Country: ES | Date: 2025-09-08 | Headlines: 10...,0.065387
1,2025-09-05,ES,Country: ES | Date: 2025-09-05 | Headlines: 13...,0.068764
2,2025-09-10,ES,Country: ES | Date: 2025-09-10 | Headlines: 68...,0.076997
3,2025-09-04,ES,Country: ES | Date: 2025-09-04 | Headlines: 15...,0.077399
4,2025-09-09,ES,Country: ES | Date: 2025-09-09 | Headlines: 11...,0.083041


## 8. Extracting Top Entities

To add more context, we enrich each `(country, day)` with the **most frequent entities**:  

- **Themes** → GDELT high-level topics (*e.g., elections, protests, wildfires*).  
- **People** → prominent names appearing in the news, with noise removed (*e.g., Facebook, Twitter filtered out*).  

👉 Why this matters:  
- AI briefings can mention **specific actors and drivers**, not just vague events.  
- The MoodMap becomes **explainable** — users see *who* and *what* shaped the sentiment.  
- Enables richer downstream analysis (e.g., tracking which leaders are linked to negative news).  

✅ Output table: **`daily_top_entities`**  
Each row contains:  
`{ event_date, country, top_themes[], top_people[] }`  


In [50]:
# Create table of top entities (themes + people)

top_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.daily_top_entities`
PARTITION BY event_date
CLUSTER BY country AS
WITH
-- explode arrays
e AS (
  SELECT
    event_date,
    country,
    LOWER(SPLIT(t, ',')[OFFSET(0)]) AS theme,    -- keep name only
    LOWER(SPLIT(p, ',')[OFFSET(0)]) AS person
  FROM `{project_id}.world_mood.gdelt_events_enriched`,
  UNNEST(themes) t,
  UNNEST(persons) p
  WHERE event_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
),
-- simple noise filter for persons
clean_people AS (
  SELECT *
  FROM e
  WHERE person IS NOT NULL
    AND LENGTH(person) >= 3
    AND person NOT IN ('facebook','twitter','instagram','linkedin','whatsapp','youtube')
),
theme_counts AS (
  SELECT event_date, country, theme, COUNT(*) AS c
  FROM e
  WHERE theme IS NOT NULL
  GROUP BY 1,2,3
),
person_counts AS (
  SELECT event_date, country, person, COUNT(*) AS c
  FROM clean_people
  GROUP BY 1,2,3
),
top_themes AS (
  SELECT event_date, country,
         ARRAY_AGG(STRUCT(theme, c) ORDER BY c DESC LIMIT 10) AS top_themes
  FROM theme_counts
  GROUP BY 1,2
),
top_people AS (
  SELECT event_date, country,
         ARRAY_AGG(STRUCT(person, c) ORDER BY c DESC LIMIT 10) AS top_people
  FROM person_counts
  GROUP BY 1,2
)
SELECT
  t.event_date,
  t.country,
  top_themes,
  top_people
FROM top_themes t
LEFT JOIN top_people p
USING (event_date, country);
"""
client.query(top_query).result()

## 9. Building Historical Analogs

We go beyond “what happened today” by connecting news to **historical precedents**.  
Using **vector search** over `news_embeddings`, each `(country, day)` is linked to its most semantically similar **past events**:  

- For every `(country, day)`, we find the **K nearest neighbors** in embedding space.  
- We store both the **structured analog list** and a **compact text snippet**.  

👉 Why this matters:  
- Helps users understand *patterns* (“today’s protests resemble those of last month”).  
- Makes summaries forward-looking — briefings can reference **history as context**.  
- Adds a unique layer of insight beyond sentiment scores.  

✅ Output table: **`daily_analogs`**  
Each row contains:  
`{ event_date, country, analogs[], analogs_txt }`


In [51]:
TOP_N = 80
today_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.today_for_analogs` AS
WITH latest AS (
  SELECT MAX(event_date) AS d
  FROM `{project_id}.world_mood.daily_country_topics`
)
SELECT
  dct.event_date,
  dct.country
FROM `{project_id}.world_mood.daily_country_topics` dct
JOIN latest ON dct.event_date = latest.d
ORDER BY dct.headline_count DESC
LIMIT {TOP_N};
"""

client.query(today_query).result()
print("✅ today_for_analogs table created")


✅ today_for_analogs table created


In [52]:
ANALOG_SNIP_CHARS = 400
ANALOG_TOPK = 5 
rows = client.query(f"SELECT country, event_date FROM `{project_id}.world_mood.today_for_analogs`").result()
all_results = []

for row in rows:
    country = row.country
    event_date = row.event_date

    # Run fn_similar_to_day
    sim_query = f"""
    SELECT
      '{event_date}' AS event_date,
      '{country}' AS country,
      event_date AS past_date,
      SUBSTR(REGEXP_REPLACE(topic_doc, r'https?://\\S+', ''), 1, {ANALOG_SNIP_CHARS}) AS snippet,
      distance
    FROM `{project_id}.world_mood.fn_similar_to_day`('{country}', DATE '{event_date}')
    WHERE event_date < DATE '{event_date}'
    ORDER BY distance ASC
    LIMIT {ANALOG_TOPK}
    """
    sim_rows = client.query(sim_query).result()
    for r in sim_rows:
        all_results.append((r.event_date, r.country, r.past_date, r.snippet, r.distance))


In [53]:

# Build the dataframe
df = pd.DataFrame(all_results, columns=["event_date", "country", "past_date", "snippet", "distance"])

# ✅ Fix: convert to datetime.date
df["event_date"] = pd.to_datetime(df["event_date"]).dt.date
df["past_date"] = pd.to_datetime(df["past_date"]).dt.date

# BigQuery table ID
table_id = f"{project_id}.world_mood.daily_analogs_flat"

# Set the schema
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("event_date", "DATE"),
        bigquery.SchemaField("country", "STRING"),
        bigquery.SchemaField("past_date", "DATE"),
        bigquery.SchemaField("snippet", "STRING"),
        bigquery.SchemaField("distance", "FLOAT"),
    ],
    write_disposition="WRITE_TRUNCATE",
)

# Load the data
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()

print("✅ daily_analogs_flat loaded")


✅ daily_analogs_flat loaded


In [54]:


final_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.daily_analogs`
PARTITION BY event_date
CLUSTER BY country AS
WITH ranked AS (
  SELECT
    event_date,
    country,
    ARRAY_AGG(
      STRUCT(past_date, snippet, distance)
      ORDER BY distance ASC
      LIMIT {ANALOG_TOPK}
    ) AS analogs
  FROM `{project_id}.world_mood.daily_analogs_flat`
  GROUP BY event_date, country
)
SELECT
  event_date,
  country,
  analogs,
  ARRAY_TO_STRING(
    ARRAY(
      SELECT
        CONCAT(CAST(a.past_date AS STRING), ': ', a.snippet)
      FROM UNNEST(analogs) AS a
      ORDER BY a.distance ASC
    ),
    '\\n- '
  ) AS analogs_txt
FROM ranked;
"""

job = client.query(final_query)
job.result()
print("✅ daily_analogs created")


✅ daily_analogs created


## 10. AI Briefings

This is the **core feature of the pipeline**: turning raw GDELT data into **human-readable daily summaries**.  

- **LLM Prompt design** → Always structured into 4 sections:  
  - *What happened*  
  - *Key drivers*  
  - *Impact*  
  - *What to watch next*  

- **Grounding the model** with factual signals:  
  - Top headlines, tone stats (avg/min/max), themes, and key people.  
  - Historical analogs from semantic search to highlight **precedents**.  

👉 Why this matters:  
- Converts noisy raw data into **decision-ready intelligence**.  
- Ensures outputs are concise (≤90 words) and **action-oriented**.  
- Gives readers both context (*drivers, impact*) and foresight (*what to monitor next*).  

✅ Output table: **`daily_briefings`**  
Each row contains:  
`{ event_date, country, briefing_text }`


In [55]:
CONTEXT_CHARS = 700
MAX_TOKENS = 300
TEMP = 0.2

brief_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.daily_briefings`
PARTITION BY event_date AS

WITH latest AS (
  SELECT MAX(event_date) AS d
  FROM `{project_id}.world_mood.daily_country_topics`
),

to_summarize AS (
  SELECT
    dct.event_date,
    dct.country,
    dct.headline_count,
    SUBSTR(REGEXP_REPLACE(dct.topic_doc, r'https?://\\S+', ''), 1, {CONTEXT_CHARS}) AS ctx,
    -- Top entities
    ARRAY_TO_STRING(ARRAY(SELECT t.theme  FROM UNNEST(ent.top_themes) t), ', ') AS themes_top,
    ARRAY_TO_STRING(ARRAY(SELECT p.person FROM UNNEST(ent.top_people) p), ', ') AS people_top,
    analogs.analogs_txt
  FROM `{project_id}.world_mood.daily_country_topics` dct
  JOIN latest ON dct.event_date = latest.d
  LEFT JOIN `{project_id}.world_mood.daily_top_entities` ent
    ON dct.event_date = ent.event_date AND dct.country = ent.country
  LEFT JOIN `{project_id}.world_mood.daily_analogs` analogs
    ON dct.event_date = analogs.event_date AND dct.country = analogs.country
  ORDER BY dct.headline_count DESC
  LIMIT {TOP_N}
),

normalized AS (
  SELECT
    event_date,
    country,
    headline_count,
    ctx,
    COALESCE(analogs_txt, 'None') AS analogs_txt,
    INITCAP(
      TRIM(
        REGEXP_REPLACE(
          REGEXP_REPLACE(
            REGEXP_REPLACE(
              REGEXP_REPLACE(
                REGEXP_REPLACE(
                  LOWER(COALESCE(themes_top,'')),
                  r'(?i)uspec_', ''
                ),
                r'(?i)wb_\\d+_', ''
              ),
              r'(?i)tax_fncact_', 'tax '
            ),
            r'(?i)crisislex_[^, ]+', 'crisis response'
          ),
          r'_', ' '
        )
      )
    ) AS themes_nice,
    INITCAP(
      TRIM(
        REGEXP_REPLACE(COALESCE(people_top,''), r'(?i)\\bLos Angeles\\b,?\\s*', '')
      )
    ) AS people_nice
  FROM to_summarize
),

llm AS (
  SELECT
    s.event_date,
    s.country,
    t.ml_generate_text_llm_result AS llm_text,
    TO_JSON_STRING(t) AS llm_raw
  FROM normalized s
  JOIN ML.GENERATE_TEXT(
    MODEL `{project_id}.world_mood.gen_model`,
    (
      SELECT
        event_date,
        country,
        CONCAT(
          'You are a news analyst. Country is an ISO code.', CHR(10),
          'Fill in this EXACT template with ≤90 words, no intro/outro:', CHR(10),
          '[What happened] ', CHR(10),
          '[Key drivers] ', CHR(10),
          '[Impact] ', CHR(10),
          '[Watch next] ', CHR(10),
          'Rules:', CHR(10),
          '- Only summarize events for ISO code ', country, '.', CHR(10),
          '- Use at least TWO items from "Top themes (readable)" and at least ONE name from "Top people".', CHR(10),
          '- Do NOT output raw taxonomy tokens (e.g., uspec_politics_general1 / crisislex_*). Use natural English phrases only.', CHR(10),
          '- Do NOT mention event codes. Avoid boilerplate like "diverse/broad range of events".', CHR(10),
          '- Include at least one concrete number if available (e.g., headline count, tone).', CHR(10),
          '- ALWAYS include all 4 sections: What happened, Key drivers, Impact, and Watch next.', CHR(10),
          '- Do NOT stop before completing the full template.', CHR(10),
          '- Limit each section to 1–2 sentences. Avoid speculation. Be concise and specific.', CHR(10),


          'Top themes (readable): ', COALESCE(themes_nice, 'none'), CHR(10),
          'Top people: ', COALESCE(people_nice, 'none'), CHR(10),
          'Context: ', ctx, CHR(10),
          'Relevant past events to consider (analog history):', CHR(10),
          analogs_txt
        ) AS prompt
      FROM normalized
    ),
    STRUCT({TEMP} AS temperature, {MAX_TOKENS} AS max_output_tokens, TRUE AS flatten_json_output)
  ) AS t
  ON s.event_date = t.event_date AND s.country = t.country
),

fallback AS (
  SELECT
    event_date,
    country,
    llm_text,
    SAFE.PARSE_JSON(llm_raw) AS j
  FROM llm
),

final AS (
  SELECT
    event_date,
    country,
    COALESCE(
      llm_text,
      JSON_VALUE(j, '$.ml_generate_text_result.candidates[0].content.parts[0].text'),
      JSON_VALUE(j, '$.ml_generate_text_result.candidates[0].content[0].text'),
      JSON_VALUE(j, '$.predictions[0].content'),
      JSON_VALUE(j, '$.text')
    ) AS briefing_text
  FROM fallback
)

SELECT *
FROM final
WHERE briefing_text IS NOT NULL;
"""

job = client.query(brief_query)
job.result()
print("✅ daily_briefings created (with analog enrichment)")


✅ daily_briefings created (with analog enrichment)


## 11. Daily MoodMap Scoring

We transform raw sentiment signals into a **single, comparable score per country/day**.  

- **AI sentiment (LLM):**  
  The model reads each country’s daily briefing and assigns a score from **−1 (very negative)** to **+1 (very positive)**.  

- **Blended mood score:**  
  Combines the **LLM sentiment** with normalized **GDELT avg_tone**, ensuring balance between narrative summaries and raw data.  

- **Hover-ready summaries:**  
  A second LLM pass generates **concise, neutral 1-sentence summaries** for map visualization tooltips.  

👉 Why this matters:  
- Converts scattered sentiment into a **global, standardized metric**.  
- Enables direct comparison across countries.  
- Supports **dashboard-ready visualization** (color-coded maps).  

✅ Output table: **`daily_moodmap`**  
Each row includes:  
`{ event_date, country, mood_score, top_themes[], summary_ref }`


In [56]:
moodmap_query = f"""
CREATE OR REPLACE TABLE `{project_id}.world_mood.daily_moodmap`
PARTITION BY event_date AS

WITH
latest_date AS (
  SELECT MAX(event_date) AS d
  FROM `{project_id}.world_mood.daily_briefings`
),

joined_data AS (
  SELECT
    b.event_date,
    b.country,
    b.briefing_text,
    g.avg_tone,
    ent.top_themes
  FROM `{project_id}.world_mood.daily_briefings` b
  JOIN latest_date l ON b.event_date = l.d
  LEFT JOIN `{project_id}.world_mood.daily_country_topics` g
    ON b.event_date = g.event_date AND b.country = g.country
  LEFT JOIN `{project_id}.world_mood.daily_top_entities` ent
    ON b.event_date = ent.event_date AND b.country = ent.country
),

llm_outputs AS (
  SELECT
    s.event_date,
    s.country,
    s.avg_tone,
    s.top_themes,
    s.briefing_text,
    t.ml_generate_text_llm_result AS llm_text,
    TO_JSON_STRING(t) AS llm_raw
  FROM joined_data s
  JOIN ML.GENERATE_TEXT(
    MODEL `{project_id}.world_mood.gen_model`,
    (
      SELECT
        event_date,
        country,
        CONCAT(
          'You are a sentiment analysis model. Read the news briefing and respond with a single number between -1 (very negative) and 1 (very positive).\\n',
          'Briefing:\\n',
          briefing_text, '\\n',
          'Sentiment score:'
        ) AS prompt
      FROM joined_data
    ),
    STRUCT({TEMP} AS temperature, {MAX_TOKENS} AS max_output_tokens, TRUE AS flatten_json_output)
  ) AS t
  ON s.event_date = t.event_date AND s.country = t.country
),

parsed_scores AS (
  SELECT
    event_date,
    country,
    avg_tone,
    top_themes,
    briefing_text,
    SAFE_CAST(REGEXP_EXTRACT(llm_text, r"-?\\d+\\.\\d+") AS FLOAT64) AS sentiment_score
  FROM llm_outputs
),

blended_scores AS (
  SELECT
    event_date,
    country,
    ROUND((
      SAFE_CAST(sentiment_score AS FLOAT64) +
      SAFE_CAST(avg_tone AS FLOAT64) / 10
    ) / 2, 4) AS mood_score,
    top_themes,
    briefing_text
  FROM parsed_scores
),

-- 🆕 Add short summary for visualization hovers
short_summary AS (
  SELECT
    s.event_date,
    s.country,
    s.mood_score,
    s.top_themes,
    s.briefing_text,
    t.ml_generate_text_llm_result AS summary_ref
  FROM blended_scores s
  JOIN ML.GENERATE_TEXT(
    MODEL `{project_id}.world_mood.gen_model`,
    (
      SELECT
        event_date,
        country,
        CONCAT(
  'Summarize today’s news mood for the ISO country code "', country, '" (treat this as a COUNTRY, not a US state).',
  ' Write ONE sentence (≤25 words).',
  ' Do not use labels like [Impact] or [What happened].',
  ' Keep it concise, neutral, and hover-friendly.'
)
 AS prompt
      FROM blended_scores
    ),
    STRUCT({TEMP} AS temperature, 60 AS max_output_tokens, TRUE AS flatten_json_output)
  ) AS t
  ON s.event_date = t.event_date AND s.country = t.country
)

SELECT *
FROM short_summary
WHERE mood_score IS NOT NULL;
"""

job = client.query(moodmap_query)
job.result()
print("✅ daily_moodmap created (with normalized tone, sentiment score, and short hover summary)")


✅ daily_moodmap created (with normalized tone, sentiment score, and short hover summary)


In [57]:
# Data Peek
df = client.query(f"""
  SELECT event_date, country, mood_score, top_themes, summary_ref
  FROM `{project_id}.world_mood.daily_moodmap`
  WHERE event_date = (SELECT MAX(event_date) FROM `{project_id}.world_mood.daily_moodmap`)
  ORDER BY mood_score DESC
  LIMIT 10
""").to_dataframe()
df.head(10)


,event_date,country,mood_score,top_themes,summary_ref
0,2025-09-10,KZ,0.1646,"[{'theme': 'tax_fncact_minister', 'c': 1643}, ...",Kazakhstan faces economic challenges amid geop...
1,2025-09-10,TW,0.0645,"[{'theme': 'ungp_forests_rivers_oceans', 'c': ...",Tensions with China remain high amid ongoing m...
2,2025-09-10,VM,0.0047,"[{'theme': 'general_government', 'c': 3611}, {...",I am unable to find any news related to the co...
3,2025-09-10,AJ,-0.1077,"[{'theme': 'uspec_politics_general1', 'c': 491...","I cannot provide a summary for ""AJ"" as an ISO ..."
4,2025-09-10,SN,-0.2025,"[{'theme': 'crisislex_crisislexrec', 'c': 668}...",Senegal faces political tensions amid ongoing ...
5,2025-09-10,RO,-0.2253,"[{'theme': 'drones', 'c': 3841}, {'theme': 'ta...",Romania faces political tensions amid ongoing ...
6,2025-09-10,NI,-0.2639,"[{'theme': 'leader', 'c': 36543}, {'theme': 't...",Nicaragua faces continued political tensions a...
7,2025-09-10,EG,-0.2652,"[{'theme': 'tax_terror_group_hamas', 'c': 7016...",Egypt faces economic challenges amid ongoing r...
8,2025-09-10,RP,-0.2761,"[{'theme': 'wb_696_public_sector_management', ...",The Philippines faces economic challenges amid...
9,2025-09-10,DA,-0.2787,"[{'theme': 'leader', 'c': 584}, {'theme': 'usp...","I cannot provide a summary for ""DA"" as an ISO ..."


## 12. Choropleth Visualization

We transform the `daily_moodmap` table into an **interactive world map**:

- 🌍 **Global choropleth:** each country is colored by its `mood_score`  
  (−1 = red 😡, 0 = neutral 😐, +1 = green 😃).  
- 🖱️ **Hover tooltips:** display the **country name**, numeric mood score, and the **1-sentence AI summary**.  
- 🎨 **Presentation-ready design:**  
  - Emoji-labeled color scale for readability.  
  - Centered title with subtitle: *“Data from GDELT + BigQuery AI.”*  
  - Clean margins and polished layout for dashboards.

👉 This is the **final demo output**: a global snapshot of sentiment, where users can instantly see  
which countries are facing negative or positive news climates, and why.  

✅ Visualization powered by **Plotly Express** directly on top of BigQuery data.


In [58]:
!pip install pycountry

In [59]:

# Query daily_moodmap for the latest day
latest_sql = f"""
SELECT *
FROM `{project_id}.world_mood.daily_moodmap`
WHERE event_date = (SELECT MAX(event_date) FROM `{project_id}.world_mood.daily_moodmap`)
"""
today_df = client.query(latest_sql).to_dataframe()


In [60]:
import pycountry

def iso2_to_iso3(iso2):
    try:
        return pycountry.countries.get(alpha_2=iso2).alpha_3
    except:
        return None

today_df["iso3"] = today_df["country"].apply(iso2_to_iso3)


In [61]:
import plotly.express as px

fig = px.choropleth(
    today_df,
    locations="iso3",
    locationmode="ISO-3",
    color="mood_score",
    color_continuous_scale=px.colors.diverging.RdYlGn,
    range_color=(-1, 1),
    title="🌍 Global News MoodMap"
)

# Custom hover: clean, structured, but full summary
fig.update_traces(
    hovertemplate=(
        "<b>%{hovertext}</b><br>" +                # Country name
        "Mood Score: <b>%{z:.2f}</b><br><br>" +    # Numeric score, bold
        "<b>Summary:</b><br>%{customdata[0]}"      # Full summary (line break before it)
    ),
    hovertext=today_df["country"],                 # human-readable name
    customdata=today_df[["summary_ref"]]           # pass full summary
)

# Prettier layout
fig.update_layout(
    title=dict(
        text="🌍 Global News MoodMap<br><sup>Data from GDELT + BigQuery AI</sup>",
        x=0.5,
        xanchor="center"
    ),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        coastlinecolor="LightGray",
        projection_type="natural earth",
        showcountries=True,
        countrycolor="white"
    ),
    coloraxis_colorbar=dict(
        title="Mood",
        tickvals=[-1, -0.5, 0, 0.5, 1],
        ticktext=["😡 Very Negative", "😟 Negative", "😐 Neutral", "🙂 Positive", "😃 Very Positive"],
        len=0.75
    ),
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()


## Conclusion  

This project showed how **BigQuery AI + GDELT** can transform the overwhelming flood of global news into an **actionable Global MoodMap**:  
- 🌍 **Daily curation**: millions of raw events distilled into per-country topic documents.  
- 🧠 **Embeddings + vector search**: retrieval by **meaning**, not just keywords.  
- ✍️ **LLM-powered briefings**: complex events summarized into **clear, structured insights**.  
- 📊 **Mood scoring**: blended numeric tone with AI sentiment to give each country a comparable index.  
- 🗺️ **Interactive visualization**: a choropleth map where users can explore global sentiment in one glance.  

This end-to-end workflow demonstrates how noisy, unstructured event streams can become **interpretable intelligence** that supports faster, better decisions.  

---

### Next Steps  

To evolve this prototype into a more powerful decision-support system:  

1. **Extend time horizon** → Generate `daily_briefings` and `daily_moodmap` for **14–30 days** to capture trends and spikes.  
2. **Scrape source URLs** → Enrich topic documents with **snippets or article text** for deeper context.  
3. **Regional & sector lenses** → Aggregate sentiment by **regions (e.g., EU, Asia)** or by **domains (energy, finance, health)**.  
4. **Real-time dashboards** → Deploy to **Streamlit or Looker Studio** for live interactive monitoring.  
5. **Alerting & signals** → Trigger alerts when a country’s mood shifts sharply (unrest, crisis, or sudden positivity).  

---

👉 With these extensions, the Global MoodMap could evolve from a hackathon demo into a **real-world monitoring platform** for analysts, investors, and policymakers.  
